In [ ]:
# Create some essential folder
!mkdir auth
!mkdir datasets

In [ ]:
# Download data from Google Storage

import pathlib
from google.cloud import storage
from google.oauth2 import service_account

class GCSClient:
    def __init__(self, gcs_authen_file_path: str):
        """
            Initial the Google cloud client with credential
        Args:
            gcs_authen_file_path: a file name in ./auth folder
            abstract_logger: the logger
        """
        self.__gcs_credentials = service_account.Credentials.from_service_account_file(
            gcs_authen_file_path,
            scopes=["https://www.googleapis.com/auth/cloud-platform"],
        )
        self.storage_client = storage.Client(
            credentials=self.__gcs_credentials,
            project=self.__gcs_credentials.project_id,
        )

    def download_gcs_file(self, bucket_name: str, source_blob_name: str, dest_file_path: str):
        """
            Install GCS file blob to local folder file
        Args:
            bucket_name: The ID of GCS bucket. Ex: "your-bucket-name"
            source_blob_name: The ID of GCS object which is file. Ex: "storage-object-name"
            dest_file_path: The path to which the local file should be downloaded. Ex: "local/path/to/file"
        """
        try:
            bucket = self.storage_client.bucket(bucket_name)

            # Construct a client side representation of a blob.
            # Note `Bucket.blob` differs from `Bucket.get_blob` as it doesn't retrieve
            # any content from Google Cloud Storage. As we don't need additional data,
            # using `Bucket.blob` is preferred here.
            blob = bucket.blob(source_blob_name)
            blob.download_to_filename(dest_file_path)
            print(
                "Downloaded storage object {} from bucket {} to local file {}.".format(
                    source_blob_name, bucket_name, dest_file_path
                )
            )
        except Exception as py_exc:
            print(f"Download from GCS to local file system - {str(py_exc)}")

    def download_gcs_folder(self, bucket_name: str, directory_of_blob: str, destination_folder_path: str):
        """
            Download GCS folder blob to local folder path
        Args:
            bucket_name: The ID of GCS bucket name.Ex: "your-bucket-name"
            directory_of_blob: The ID of GCS object which is folder. Ex: "storage"
            destination_folder_path: The name of the destination project
        """
        try:
            blobs_in_bucket = self.storage_client.list_blobs(bucket_name, prefix=directory_of_blob)
            for blob in blobs_in_bucket:
                local_filename = (pathlib.Path(destination_folder_path) / blob.name.split("/")[-1]).resolve()
                blob.download_to_filename(filename=str(local_filename))
                print(
                    "Downloaded storage object {}/* from bucket {} to local file {}.".format(
                        directory_of_blob, bucket_name, local_filename
                    )
                )
        except Exception as py_exc:
            print(f"Download folder blob from GCS - Failed: {py_exc}")

    def upload_gcs_file(self, bucket_name: str, source_file_path: str, dest_blob_path: str):
        """
            Upload a file to GCS bucket
        Args:
            bucket_name: The ID of GCS bucket name. Ex: "your-bucket-name"
            source_file_path: The file path of source. Ex: "local/path/to/file"
            dest_bucket_file: The ID of GCS object which is file uploaded. Ex: "storage"
        """
        try:
            bucket = self.storage_client.bucket(bucket_name)
            bucket.blob(dest_blob_path).upload_from_filename(source_file_path)
            print(f"File {source_file_path} uploaded to {dest_blob_path}.")
        except Exception as py_exc:
            print(f"Upload file to GCS blob - Failed: {py_exc}")

    def upload_gcs_folder(self, bucket_name: str, source_folder_path: str, dest_folder_bucket: str):
        """
            Upload a folder has only files to GCS bucket
        Args:
            bucket_name: The ID of GCS bucket name. Ex: "your-bucket-name"
            source_folder_path:  The folder path of source
            dest_folder_bucket: The ID of GCS object which is folder path. Ex "your-bucket-name/path/to/GCS/folder"
        """
        try:
            bucket = self.storage_client.bucket(bucket_name)
            dir_recursive_paths = pathlib.Path(source_folder_path)
            for path in dir_recursive_paths.iterdir():
                if path.is_file():
                    bucket.blob(dest_folder_bucket).upload_from_filename(dest_folder_bucket)
                    print(f"File {source_folder_path} uploaded to {dest_folder_bucket}")
        except Exception as py_exc:
            print(f"Upload the folder of file system to GCS - Failed: {py_exc}")


In [ ]:
# Download dataset from GCS
pwd = pathlib.Path()
gcs_authen_path = pwd / "auth" / "kiotviet-connect-sr.json"
diginetica_fs_path = pwd / "dataset-train-diginetica.zip"
yoochoose_fs_path = pwd / "yoochoose-data.7z"

gcsClient = GCSClient(gcs_authen_file_path=gcs_authen_path)

# Download diginetica dataset
gcsClient.download_gcs_file(
    bucket_name="bi_recommendation_hub_storage",
    source_blob_name="recommend/session-based/dataset-train-diginetica.zip",
    dest_file_path=diginetica_fs_path
)

# Download yoochoose
gcsClient.download_gcs_file(
    bucket_name="bi_recommendation_hub_storage",
    source_blob_name="recommend/session-based/yoochoose-data.7z",
    dest_file_path=yoochoose_fs_path
)

Downloaded storage object recommend/session-based/dataset-train-diginetica.zip from bucket bi_recommendation_hub_storage to local file dataset-train-diginetica.zip.
Downloaded storage object recommend/session-based/yoochoose-data.7z from bucket bi_recommendation_hub_storage to local file yoochoose-data.7z.


In [ ]:
%%shell
mkdir /content/datasets/diginetica
unzip /content/dataset-train-diginetica.zip -d /content/datasets/diginetica
echo "======================"
mkdir /content/datasets/yoochoose
7z x /content/yoochoose-data.7z -o/content/datasets/yoochoose

Archive:  /content/dataset-train-diginetica.zip
  inflating: /content/datasets/diginetica/train-clicks.csv  
  inflating: /content/datasets/diginetica/train-purchases.csv  
  inflating: /content/datasets/diginetica/train-item-views.csv  
  inflating: /content/datasets/diginetica/train-queries.csv  
  inflating: /content/datasets/diginetica/products.csv  
  inflating: /content/datasets/diginetica/product-categories.csv  

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 287211932 bytes (274 MiB)

Extracting archive: /content/yoochoose-data.7z
--
Path = /content/yoochoose-data.7z
Type = 7z
Physical Size = 287211932
Headers Size = 255
Method = LZMA:24
Solid = +
Blocks = 2

  0%      0% - yoochoose-buys.dat

4.1. Dataset
yoochoose là tập dữ liệu được cung cấp từ cuộc thi RecSys Challange 2015. Dữ liệu bao gồm tập hợp các thông tin về các click event trong từng session tương ứng. Trong các events sẽ bao gồm buy event giúp xác định các sự kiện mua hàng. Mục tiêu là dự báo khách hàng có khả năng mua hàng sắp tới không và nếu mua hàng thì mua sản phẩm nào?

Bộ dữ liệu bao gồm 3 files chính:

Các files huấn luyện:

1 . yoochoose-clicks.dat: Dữ liệu về các click events. Mỗi một dòng bao gồm các trường sau đây:

Session ID: Id của session. Trong một session có thể có 1 hoặc nhiều lượt clicks.
Timestamp: Thời điểm diễn ra click.
Item ID: Id dùng để xác định item.
Category: Nhóm category của item.
2 . yoochoose-buys.dat: Dữ liệu về buy events. Mỗi một dòng bao gồm các trường sau đây:

Session ID: Id của session. Trong một session có thể có 1 hoặc nhiều lượt mua sắm.
Timestamp: Thời điểm diễn ra hành vi mua sắm.
Item ID: Id của item.
Price: Gía của sản phẩm.
Quantity: Số lượng sản phẩm được mua.
Do các trong các session sẽ một số session có các event buying nên số lượng các session trong file yoochoose-buys.dat nhiều hơn so với yoochoose-clicks.dat và mọi session trong file yoochoose-buys.dat sẽ chứa trong file yoochoose-clicks.dat. Một session có thể rất ngắn (vài phút) hoặc rất dài (vài giờ) và theo đó số lượng event cũng tương ứng ít hoặc nhiều tùy vào hoạt động của user.

File kiểm tra:

3 . yoochoose-test.dat: Cấu trúc tương tự như file yoochoose-clicks.dat của huấn luyện. Bao gồm các trường: Session ID, Timestamp, Item ID, Category. Mục tiêu của chúng ta là cần dự báo xem trong các session của tập test có event buying hay không và nếu có thì list các Item ID được mua là gì?



4.1.1. Khởi tạo dữ liệu

In [ ]:
import os
import pandas as pd
from datetime import datetime
import time

filepath = '/content/datasets/yoochoose/yoochoose-clicks.dat'
dataset = pd.read_csv(filepath, names = ['SessionId', 'DateTime', 'ItemId', 'Category'])
dataset['DateTime'] = dataset['DateTime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'))
dataset['Timestamp'] = dataset['DateTime'].apply(lambda x: x.strftime('%s.%f'))
dataset = dataset.sort_values(by = ['SessionId', 'DateTime'], ascending = True)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Thống kê số lượt xuất hiện và lọc ra các ItemId có trên 5 lượt xuất hiện
df_item_count = dataset[['ItemId', 'SessionId']].groupby('ItemId').count().sort_values(by = 'SessionId', ascending = False)
df_item_count.columns = ['CountItemId']
df_item_count_5 = df_item_count[df_item_count['CountItemId'] < 5]
# Lọc khỏi dataset những ItemId có ít hơn 5 lượt xuất hiện
dataset = dataset[~dataset['ItemId'].isin(list(df_item_count_5.index))]

4.1.2. Phân chia dữ liệu train/test
Từ tập dữ liệu train ta sẽ tách ra 7 ngày cuối cùng làm dữ liệu test. Đối với dữ liệu còn lại để việc huấn luyện được nhanh hơn thì chúng ta sẽ chỉ dữ lại 1/4 số lượng các session cho huấn luyện.

Khi đó dữ liệu test sẽ có cấu trúc tương tự như train, mỗi session bao gồm các itemIds được sắp xếp theo thứ tự thời gian. Từ list các itemId liền trước ta cần dự báo itemId tiếp theo có khả năng được click. Các dữ liệu trên test được xem như là session mới hoàn toàn và được sử dụng để kiểm tra mức độ dự báo chuẩn xác của mô hình được huấn luyện từ tập dữ liệu train.

Bên dưới ta sẽ thực hành phân chia train/test cho mô hình.

In [ ]:
from datetime import timedelta
# Phân chia tập train/test sao cho tập test là 7 ngày gần đây nhất và train là dữ liệu còn lại
maxdate = dataset['DateTime'].max()
mindate7 = maxdate - timedelta(days = 7)
test = dataset[dataset['DateTime'] >= mindate7]
dataset = dataset[dataset['DateTime'] <= mindate7]

Lấy ra ngẫu nhiên 1/4 số lượng các session cho huấn luyện.

In [ ]:
import numpy as np

# list các sessionIds
sessIds = list(dataset['SessionId'].unique())
# Lấy ngẫu nhiên 1/4 số lượng các session
n_filter = int(len(sessIds)/4)
np.random.shuffle(sessIds)
sessIdsFilter = sessIds[:n_filter]
# Lựa chọn các 1/4 session làm dataset train (dữ liệu này bao gồm cả train và validation)
# Set index là sessionId để filter nhanh hơn
dataset.set_index('SessionId', inplace=True)
dataset_filter = dataset[dataset.index.isin(sessIdsFilter)]
dataset_filter = dataset_filter.reset_index()

Khởi tạo chuỗi các itemId sắp xếp theo thời gian trên mỗi session. Mỗi một itemId sẽ tương ứng với giá trị thời gian của nó. Cấu trúc của các train_sess, test_sess có dạng:

['sessionId': {'itemId1':'Timestamp1', ..., 'itemId_n':'Timestamp_n'}]

In [ ]:
# Lấy ra dictionary có dạng {SessionId:{ItemId1:Timestamp1, ItemId2:Timestamp2, ...}}
train_sess = dataset_filter[['SessionId', 'ItemId', 'Timestamp']].groupby('SessionId').apply(lambda x: dict(zip(x['ItemId'], x['Timestamp'])))
test_sess = test[['SessionId', 'ItemId', 'Timestamp']].groupby('SessionId').apply(lambda x: dict(zip(x['ItemId'], x['Timestamp'])))

In [ ]:
print(train_sess[:10])

SessionId
1     {214536502: '1396867869.277000', 214536500: '1...
6     {214701242: '1396803500.848000', 214826623: '1...
16                     {214684093: '1396704053.092000'}
22    {214837485: '1396747860.419000', 214837487: '1...
23                     {214718203: '1396598208.356000'}
26    {214579288: '1396802575.741000', 214714790: '1...
27    {214827028: '1396856412.650000', 214827017: '1...
29    {214532036: '1396460527.324000', 214700432: '1...
37    {214826769: '1396628767.882000', 214537967: '1...
44    {214820441: '1396849890.888000', 214826897: '1...
dtype: object


In [ ]:
print(test_sess[:10])

SessionId
11255548    {214830939: '1411588414.517000', 214854444: '1...
11255549    {214716973: '1411640755.459000', 214716937: '1...
11255551                     {214545500: '1411838500.441000'}
11255552    {214853100: '1411657491.567000', 214537239: '1...
11255553    {214540020: '1411846389.986000', 214819723: '1...
11255554    {214850752: '1411600883.048000', 214686879: '1...
11255556    {214563189: '1411821070.438000', 214712051: '1...
11255557    {214834922: '1411724558.366000', 214856831: '1...
11255558                     {214854767: '1411671743.296000'}
11255559                     {214512605: '1411997055.971000'}
dtype: object


4.1.3. Preprocessing data
Ở bước này ta sẽ khởi tạo input và output cho mô hình.

Input của mô hình là list các itemIds trong quá khứ và Output là itemId ở vị trí hiện tại. Qúa trình khởi tạo các Input và Output trên một session được thực hiện tịnh tiến từ vị trí itemID đầu tiên cho đến cuối cùng.

Ví dụ: Chúng ta có sessionId = [itemId_1, itemId_2, ..., itemId_n] Như vậy sau các cặp (input, output) sẽ là:

cặp thứ 1: ([itemId_1], [itemId_2])
cặp thứ 2: ([itemId_1, itemId_2], [itemId_3])

…

cặp thứ n-1: ([itemId_1, itemId_2,..., itemId_(n-1)], [itemId_n])

In [ ]:
sessDict = {214834865: '1396808691.295000', 214706441: '1396808691.426000', 214820225: '1396808691.422000'}

def _preprocess_sess_dict(sessDict):
    sessDictTime = dict([(v, k) for (k, v) in sessDict.items()])
    sessSort = sorted(sessDictTime.items(), reverse = False)
    times = [item[0] for item in sessSort]
    itemIds = [item[1] for item in sessSort]
    inp_seq = []
    labels = []
    inp_time = []

    for i in range(len(sessSort)):
        if i >= 1:
            inp_seq += [itemIds[:i]]
            labels += [itemIds[i]]
            inp_time += [times[i]]
    return inp_seq, inp_time, labels, itemIds

inp_seq, inp_time, labels, itemIds = _preprocess_sess_dict(sessDict)
print('input sequences: ', inp_seq)
print('input times: ', inp_time)
print('targets: ', labels)

input sequences:  [[214834865], [214834865, 214820225]]
input times:  ['1396808691.422000', '1396808691.426000']
targets:  [214820225, 214706441]


In [ ]:
# Khởi tạo chuỗi input và output cho toàn bộ các session
def _preprocess_data(data_sess):
    inp_seqs = []
    inp_times = []
    labels = []
    sequences = []
    sessIds = list(data_sess.index)
    for sessId in sessIds:
        sessDict = data_sess.loc[sessId]
        inp_seq, inp_time, label, sequence = _preprocess_sess_dict(sessDict)
        inp_seqs += inp_seq
        inp_times += inp_time
        labels += label
        sequences += sequence
    return inp_seqs, inp_times, labels, sequences

train_inp_seqs, train_inp_dates, train_labs, train_sequences = _preprocess_data(train_sess)
test_inp_seqs, test_inp_dates, test_labs, test_sequences = _preprocess_data(test_sess)

train = (train_inp_seqs, train_labs)
test = (test_inp_seqs, test_labs)

print('Done.')

Done.


In [ ]:
!pwd

/content


In [ ]:
# Do kích thước dữ liệu là khá lớn nên để thuận lợi cho những lượt train sau ta nên lưu dữ liệu của train và test vào một folder là yoochoose-data-1-4.
import pickle
import os
import pathlib

pwd = pathlib.Path() 
saved_data_path = pwd / "yoochoose-data"

def _save_file(filename, obj):
    print(filename)
    with open(filename, 'wb') as fn:
        pickle.dump(obj, fn)

# Tạo folder yoochoose-data-4 để lưu dữ liệu train/test nếu chưa tồn tại
if not os.path.exists(saved_data_path):
    os.mkdir(saved_data_path)

# Lưu train/test
_save_file(saved_data_path / "train.pkl", train)
_save_file(saved_data_path / "test.pkl", test)

yoochoose-data/train.pkl
yoochoose-data/test.pkl


In [ ]:
# Ở những lượt huấn luyện sau ta chỉ cần load lại những dữ liệu train/test đã lưu tại folder yoochoose-data-4
import pickle

def _load_file(filename):
  with open(filename, 'rb') as fn:
    data = pickle.load(fn)
  return data

# Load dữ liệu train/test từ folder
train = _load_file(saved_data_path / "train.pkl")
test = _load_file(saved_data_path / "test.pkl") 


4.1.4. Khởi tạo dictionary
Chúng ta cần sử dụng dictionary để nhúng itemId của mỗi một sản phẩm thành một index sao cho mỗi chỉ số này là duy nhất đối với mỗi itemId. Từ index ta có thể tạo ra được các véc tơ one-hot dễ dàng làm đầu vào cho huấn luyện mô hình ở bước embedding.

In [ ]:
# Các token default
PAD_token = 0  # token padding cho câu ngắn

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.item2index = {}
        self.item2count = {}
        self.index2item = {PAD_token: "PAD"}
        self.num_items = 1  # số lượng mặc định ban đầu là 1 ứng với PAD_token

    def addSenquence(self, data):
        for sequence in data:
          for item in sequence:
              self.addItem(item)

    # Thêm một item vào hệ thống
    def addItem(self, item):
        if item not in self.item2index:
            self.item2index[item] = self.num_items
            self.item2count[item] = 1
            self.index2item[self.num_items] = item
            self.num_items += 1
        else:
            self.item2count[item] += 1

    # Loại các item dưới ngưỡng xuất hiện min_count
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        
        keep_items = []

        for k, v in self.item2count.items():
            if v >= min_count:
                keep_items.append(k)

        print('keep_items {} / {} = {:.4f}'.format(
            len(keep_items), len(self.item2index), len(keep_items) / len(self.item2index)
        ))

        # Khởi tạo lại từ điển
        self.item2index = {}
        self.item2count = {}
        self.index2item = {PAD_token: "PAD"}
        self.num_items = 1

        # Thêm các items vào từ điển
        for item in keep_items:
            self.addItem(item)

    # Hàm convert sequence về chuỗi các indices
    def _seqItem2seqIndex(self, x):
        return [voc.item2index[item] if item in voc.item2index else 0 for item in x]


In [ ]:
# Lấy toàn bộ list các itemIds trong các session.
from itertools import chain
seq_targets = [train[1]] + [test[1]]
sessionIds = list(chain.from_iterable(seq_targets))
sessionIds = set(sessionIds)
print('Number of sessionIds: ', len(sessionIds))

Number of sessionIds:  33750


In [ ]:
# Khởi tạo vocabullary cho bộ dữ liệu.
voc = Voc('DictItemId')
voc.addSenquence(seq_targets)

# Convert thử nghiệm một sequence itemIds
print('sequence of itemIds: ', train[0][7])
print('converted indices: ', voc._seqItem2seqIndex(train[0][7]))


sequence of itemIds:  [214821275, 214821371, 214717089, 214563337, 214706462, 214717436]
converted indices:  [913, 3, 4, 5, 6, 7]


In [ ]:
# Tiếp theo ta sẽ chuyển dữ liệu train, test từ item sang indices của item
train_x_index = [voc._seqItem2seqIndex(seq) for seq in train[0]]
test_x_index = [voc._seqItem2seqIndex(seq) for seq in test[0]]
train_y_index = voc._seqItem2seqIndex(train[1])
test_y_index = voc._seqItem2seqIndex(test[1])
train_index = (train_x_index, train_y_index)
test_index = (test_x_index, test_y_index)

Từ 2 tập dữ liệu train_index và test_index ban đầu ta sẽ phân chia thành 3 tập train/test và validation như sau:

Mỗi tập dữ liệu bao gồm 2 phần: input bao gồm chuỗi các itemId liên tiếp, output là itemId tiếp theo trong được khách hàng click.

Dữ liệu validation được rút ra từ dữ liệu train set theo tỷ lệ được qui định tại valid_portion. Phần còn lại của train set được dữ làm tập train set mới.

Dữ liệu test được tính toàn trực tiếp từ tập test set.

Đối với dữ liệu input, chuỗi dữ liệu sẽ được truncate về độ dài maxlen nếu nó vượt qua maxlen.

In [ ]:
def load_data(root='', valid_portion=0.1, maxlen=19, sort_by_len=False, train_set=None, test_set=None):
    """Load dataset từ root
    root: folder dữ liệu train, trong trường hợp train_set, test_set tồn tại thì không sử dụng train_set và test_set
    valid_portion: tỷ lệ phân chia dữ liệu validation/train
    maxlen: độ dài lớn nhất của sequence
    sort_by_len: có sort theo chiều dài các session trước khi chia hay không?
    train_set: training dataset
    test_set:  test dataset
    """
    
    # Load the dataset
    if train_set is None and test_set is None:
        path_train_data = os.path.join(root, 'train.pkl')
        path_test_data = os.path.join(root, 'test.pkl')
        with open(path_train_data, 'rb') as f1:
            train_set = pickle.load(f1)

        with open(path_test_data, 'rb') as f2:
            test_set = pickle.load(f2)

    if maxlen:
        new_train_set_x = []
        new_train_set_y = []
        # Lọc dữ liệu sequence đến maxlen
        for x, y in zip(train_set[0], train_set[1]):
            if len(x) < maxlen:
                new_train_set_x.append(x)
                new_train_set_y.append(y)
            else:
                new_train_set_x.append(x[:maxlen])
                new_train_set_y.append(y)
        train_set = (new_train_set_x, new_train_set_y)
        del new_train_set_x, new_train_set_y

        new_test_set_x = []
        new_test_set_y = []
        for xx, yy in zip(test_set[0], test_set[1]):
            if len(xx) < maxlen:
                new_test_set_x.append(xx)
                new_test_set_y.append(yy)
            else:
                new_test_set_x.append(xx[:maxlen])
                new_test_set_y.append(yy)
        test_set = (new_test_set_x, new_test_set_y)
        del new_test_set_x, new_test_set_y

    # phân chia tập train thành train và validation
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    (test_set_x, test_set_y) = test_set

    # Trả về indices thứ tự độ dài của mỗi phần tử trong seq
    def len_argsort(seq):
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))

    # Sắp xếp session theo độ dài tăng dần
    if sort_by_len:
        sorted_index = len_argsort(test_set_x)
        test_set_x = [test_set_x[i] for i in sorted_index]
        test_set_y = [test_set_y[i] for i in sorted_index]

        sorted_index = len_argsort(valid_set_x)
        valid_set_x = [valid_set_x[i] for i in sorted_index]
        valid_set_y = [valid_set_y[i] for i in sorted_index]

    train = (train_set_x, train_set_y)
    valid = (valid_set_x, valid_set_y)
    test = (test_set_x, test_set_y)
    return train, valid, test

4.2. Data Loader
Để streaming dữ liệu theo batch thì ta cần sử dụng class DataLoader của pytorch. Class này có chức năng tương tự như ImageDataGenerator trong tensorflow và keras. Nó sẽ cho phép ta sử dụng generator để sinh dữ liệu cho từng batch huấn luyện. Do đó ta sẽ có thể huấn luyện được những mô hình từ dữ liệu có kích thước lớn hơn RAM gấp rất nhiều lần. Data Loader trong pytorch sẽ sử dụng dữ liệu là các class Dataset của pytorch như bên dưới.

4.2.1. RecSysDataset
Dataset sẽ có dữ liệu hoàn toàn giống với tập train/test và validation mà ta đã phân chia ở bước trên. Chúng được tạo ra đơn thuần là để thích hợp với định dạng data được sử dụng trong quá trình khởi tạo DataLoader.

In [ ]:
from torch.utils.data import Dataset

class RecSysDataset(Dataset):
    """define the pytorch Dataset class for yoochoose and diginetica datasets.
    """
    def __init__(self, data):
        self.data = data
        print('-'*50)
        print('Dataset info:')
        print('Number of sessions: {}'.format(len(data[0])))
        print('-'*50)
        
    def __getitem__(self, index):
        session_items = self.data[0][index]
        target_item = self.data[1][index]
        return session_items, target_item

    def __len__(self):
        return len(self.data[0])

4.2.2. Hàm phụ trợ
Ngoài ra để tùy biến các dữ liệu từ Dataset, ta có thể truyền vào DataLoader thông qua tham số collate_fn một hàm số có tác dụng biến đổi dữ liệu.

Đối với mô hình này ta cũng sẽ sử dụng hàm collate_fn() như bên dưới để biến đổi data (chính là các batch) theo các bước như sau:

Sắp xếp độ dài input sequence theo độ dài list từ cao xuống thấp. Việc này sẽ giúp cho quá trình huấn luyện nhanh hơn.

Padding thêm 0 vào dữ liệu để độ dài list bằng nhau và bằng với độ dài của input sequence lớn nhất trong batch.

transpose dữ liệu từ batch_size x maxlen --> maxlen x batch_size

Cuối cùng trả về kết quả ngoài list các chuỗi sessions, list các labels còn trả thêm list các lens ghi nhận độ dài của các sesssions.

In [ ]:
import torch

def collate_fn(data):
    """
    Hàm số này sẽ được sử dụng để pad session về max length
    Args: 
      data: batch truyền vào
    return: 
      batch data đã được pad length có shape maxlen x batch_size
    """
    # Sort batch theo độ dài của input_sequence từ cao xuống thấp
    data.sort(key=lambda x: len(x[0]), reverse=True)
    lens = [len(sess) for sess, label in data]
    labels = []
    # Padding batch size
    padded_sesss = torch.zeros(len(data), max(lens)).long()
    for i, (sess, label) in enumerate(data):
        padded_sesss[i,:lens[i]] = torch.LongTensor(sess)
        labels.append(label)
    
    # Transpose dữ liệu từ batch_size x maxlen --> maxlen x batch_size
    padded_sesss = padded_sesss.transpose(0,1)
    return padded_sesss, torch.tensor(labels).long(), lens

4.3. Metric
Có 2 metrics chính để đo lường hiệu quả của mô hình recommendation đó là:

Recall@k : Metric của dữ liệu bao gồm tỷ lệ xuất hiện ground truth của itemId trong top 
 sản phẩm được suggest có xác suất lớn nhất. Tỷ lệ này cho biết khả năng khách hàng sẽ click vào một sản phẩm được suggest từ mô hình với xác suất là bao nhiêu. Do đó giá trị của nó càng lớn thì mô hình sẽ có độ chuẩn xác càng cao. Nếu recall@20 = 10% có nghĩa là nếu áp dụng mô hình để suggest ra 20 sản phẩm cho khách hàng thì khả năng họ có click vào sản phẩm là 10%.

mrr@k: Chúng ta sẽ quan tâm trong trường hợp ground truth của itemId nằm trong top 
 sản phẩm được suggest thì thứ tự là bao nhiêu? Nếu vị trí của nó càng nhỏ thì độ chính xác của mô hình càng cao. mrr@k sẽ là nghịch đảo vị trí của ground truth trong trường hợp nó nằm trong top 
 sản phẩm được suggest. Do đó mrr@k càng lớn thì mô hình càng chất lượng.

Để tính toán các metrics trên sẽ dựa trên ground truth và top 
 sản phẩm được suggest từ mô hình như bên dưới:

In [ ]:
import torch

def get_recall(indices, targets):
    """
    Tính toán chỉ số recall cho một tập hợp predictions và targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices được dự báo từ mô hình model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
    """
    # copy targets k lần để trở thành kích thước Bxk
    targets = targets.view(-1, 1).expand_as(indices)
    # so sánh targets với indices để tìm ra vị trí mà khách hàng sẽ hit.
    hits = (targets == indices).to(device)
    hits = hits.double()
    if targets.size(0) == 0:
        return 0
    # Đếm số hit
    n_hits = torch.sum(hits)
    recall = n_hits / targets.size(0)
    return recall


def get_mrr(indices, targets):
    """
    Tính toán chỉ số MRR cho một tập hợp predictions và targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices được dự báo từ mô hình model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the MRR score
    """
    tmp = targets.view(-1, 1)
    targets = tmp.expand_as(indices)
    hits = (targets == indices).to(device)
    hits = hits.double()
    if hits.sum() == 0:
      return 0
    argsort = []
    for i in np.arange(hits.shape[0]):
      index_col = torch.where(hits[i, :] == 1)[0]+1
      if index_col.shape[0] != 0:
        argsort.append(index_col.double())
    inv_argsort = [1/item for item in argsort]
    mrr = sum(inv_argsort)/hits.shape[0]
    return mrr


def evaluate(logits, targets, k=20):
    """
    Đánh giá model sử dụng Recall@K, MRR@K scores.
    Args:
        logits (B,C): torch.LongTensor. giá trị predicted logit cho itemId tiếp theo.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
        mrr (float): the mrr score
    """
    # Tìm ra indices của topk lớn nhất các giá trị dự báo.
    _, indices = torch.topk(logits, k, -1)
    recall = get_recall(indices, targets)
    mrr = get_mrr(indices, targets)
    return recall, mrr

Hàm evaluate() sẽ tính toán đồng thời cả 2 chỉ số là recall@k và mrr@k. Tham số của hàm evaluate() bao gồm:

logits: Kích thước BxC trong đó B là batch_size và C là số class. Mỗi dòng của logits là phân phối xác suất dự báo cho itemId tiếp theo.

targets: Kích thước B trong đó B là batch_size. Đây là index của itemId mà khách hàng đã click và chính là ground truth của mô hình.

Kiểm tra hàm evaluate().

In [ ]:
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

logits = torch.tensor([[0.1, 0.2, 0.7],
                       [0.4, 0.1, 0.5],
                       [0.1, 0.2, 0.7]]).to(device)

targets = torch.tensor([1, 2, 2]).to(device)

evaluate(logits = logits, targets = targets, k = 2)

(tensor(1., device='cuda:0', dtype=torch.float64),
 tensor([0.8333], device='cuda:0', dtype=torch.float64))

4.4. Model NARM
4.4.1. Các layer của NARM
Embedding Layer: Là layer nhúng giúp mã hóa các item index thành những véc tơ với chiều xác định bằng embedding_dim. Embedding Layer sẽ nhận đầu vào là một sequence của các item index. Layer này sẽ thực hiện 2 biến đổi chính:

Đầu tiên mỗi một item index sẽ được mã hóa về véc tơ one-hot (là véc tơ đơn vị có giá trị tại vị trí index = 1 và các vị trí khác bằng 0). Như vậy kích thước của véc tơ one-hot sẽ bằng kích thước của từ điển. Lấy ví dụ: Nếu từ điển có 10000 từ với index nhận giá trị từ 1-10000, khi đó một từ có index = 2 sẽ được mã hóa thành véc tơ [0, 1, 0, …, 0]. Tức vị trí số 2 của véc tơ bằng 1 và các vị trí còn lại bằng 0.

Sau khi mã hóa xong toàn bộ các từ trong câu thì input của chuỗi item index sẽ trở thành ma trận có kích thước là max_len x vocabulary_size. Khi đó để giảm chiều của dữ liệu ta sẽ sử dụng một phép chiếu linear projection để giảm chiều từ vocabulary_size về embedding_dim. Như vậy sau cùng thu được ma trận kích thước max_len x embedding_dim. Lưu ý: ta giả định là không quan tâm đến batch_size nên kích thước ma trận ở trên chưa bao gồm batch_size.

Dropout: Layer này có tác dụng tắt đi ngẫu nhiên một số lượng kết nối liên kết giữa 2 layers để giảm thiểu overfitting cho mô hình.

GRU: Layer GRU là trọng tâm của mô hình, thực hiện quá trình dự báo chuỗi. Mô hình bao gồm nhiều time step và mỗi một time step sẽ trả ra phân phối xác suất đại diện cho từ ở vị trí đó. Kết quả trả ra của layer GRU goòm 2 thành phần: (output, hidden).

Trong đó output sẽ chứa toàn bộ các hidden véc tơ tại toàn bộ các time step 
 tại layer GRU cuối cùng (chúng ta có thể chồng nhiều layers GRU nối tiếp nhau thông qua khai báo ở tham số num_layers và chúng đều đưa ra cùng một kết quả cuối cùng mà không phụ thuộc vào num_layers, kết quả trả ra sẽ được tính từ layer cuối). output sẽ có kích thước là max_len x batch_size x (n_directions*hidden_size). Sở dĩ có thêm n_direction là vì mô hình được thực hiện theo một chiều từ trái qua phải hoặc 2 chiều từ trái qua phải và từ phải qua trái. Trường hợp 2 chiều thì tại mỗi time step sẽ có 2 hidden véc tơ đại diện cho 2 chiều.

Tham số thứ 2 trả ra từ layer GRU là hidden chính là list các véc tơ hidden tại time step cuối cùng của toàn bộ các layer GRU. Như vậy hidden sẽ có kích thước là (num_layers*n_directions) x batch_size x hidden_size. Để thu được hidden state của layer GRU cuối cùng thì ta sẽ trích suất véc tơ cuối cùng (chính là công thức ht = hidden[-1] bên dưới).

Quá trình tính attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class NARM(nn.Module):
    def __init__(self, hidden_size, n_items, embedding_dim, n_layers=1, dropout=0.25):
        super(NARM, self).__init__()
        self.hidden_size = hidden_size
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(self.n_items, self.embedding_dim, padding_idx = 0)
        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        # set bidirectional = True for bidirectional
        # https://pytorch.org/docs/stable/nn.html?highlight=gru#torch.nn.GRU to get more information
        self.gru = nn.GRU(input_size = hidden_size, # number of expected feature of input x 
                          hidden_size = hidden_size, # number of expected feature of hidden state 
                          num_layers = n_layers, # number of GRU layers
                          dropout=(0 if n_layers == 1 else dropout), # dropout probability apply in encoder network
                          bidirectional=True # one or two directions.
                         )
        self.emb_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(self.embedding_dim, self.hidden_size, self.n_layers)
        self.a_1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.a_2 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.v_t = nn.Linear(self.hidden_size, 1, bias=False)
        self.ct_dropout = nn.Dropout(0.5)
        self.b = nn.Linear(self.embedding_dim, 2 * self.hidden_size, bias=False)
        self.sf = nn.Softmax()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, input_seq, input_lengths, hidden=None):
        """
        input_seq: Batch input_sequence. Shape: max_len x batch_size 
        input_lengths: Batch input lengths. Shape: batch_size
        """
        # Step 1: Convert sequence indexes to embeddings
        # shape: (max_length , batch_size , hidden_size)
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module. Padding zero when length less than max_length of input_lengths.
        # shape: (max_length , batch_size , hidden_size)
        packed = pack_padded_sequence(embedded, input_lengths)

        # Step 2: Forward packed through GRU
        # outputs is output of final GRU layer
        # hidden is concatenate of all hidden states corresponding with each time step.
        # outputs shape: (max_length , batch_size , hidden_size x num_directions)
        # hidden shape: (n_layers x num_directions , batch_size , hidden_size)
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding. Revert of pack_padded_sequence
        # outputs shape: (max_length , batch_size , hidden_size x num_directions)
        outputs, length = pad_packed_sequence(outputs)

        # Step 3: Global Encoder & Local Encoder
        # num_directions = 1 -->
        # outputs shape:(max_length , batch_size , hidden_size)
        # hidden shape: (n_layers , batch_size , hidden_size)
        # lấy hidden state tại time step cuối cùng
        ht = hidden[-1]
        # reshape outputs
        outputs = outputs.permute(1, 0, 2) # [batch_size, max_length, hidden_size]     
        c_global = ht
        # Flatten outputs thành shape: [batch_size, max_length, hidden_size]
        gru_output_flatten = outputs.contiguous().view(-1, self.hidden_size)
        # Thực hiện một phép chiếu linear projection để tạo các latent variable có shape [batch_size, max_length, hidden_size]
        q1 = self.a_1(gru_output_flatten).view(outputs.size()) 
        # Thực hiện một phép chiếu linear projection để tạo các latent variable có shape [batch_size, max_length, hidden_size]
        q2 = self.a_2(ht)
        # Ma trận mask đánh dấu vị trí khác 0 trên padding sequence.
        mask = torch.where(input_seq.permute(1, 0) > 0, torch.tensor([1.], device = self.device), torch.tensor([0.], device = self.device)) # batch_size x max_len
        # Điều chỉnh shape
        q2_expand = q2.unsqueeze(1).expand_as(q1) # shape [batch_size, max_len, hidden_size]
        q2_masked = mask.unsqueeze(2).expand_as(q1) * q2_expand # batch_size x max_len x hidden_size
        # Tính trọng số alpha đo lường similarity giữa các hidden state 
        alpha = self.v_t(torch.sigmoid(q1 + q2_masked).view(-1, self.hidden_size)).view(mask.size()) # batch_size x max_len
        alpha_exp = alpha.unsqueeze(2).expand_as(outputs) # batch_size x max_len x hidden_size
        # Tính linear combinition của các hidden state
        c_local = torch.sum(alpha_exp * outputs, 1) # (batch_size x hidden_size)

        # Véc tơ combinition tổng hợp
        c_t = torch.cat([c_local, c_global], 1) # batch_size x (2*hidden_size) 
        c_t = self.ct_dropout(c_t)
        # Tính scores

        # Step 4: Decoder
        # embedding cho toàn bộ các item
        item_indices = torch.arange(self.n_items).to(device) # 1 x n_items
        item_embs = self.embedding(item_indices) # n_items x embedding_dim
        # reduce dimension by bi-linear projection
        B = self.b(item_embs).permute(1, 0) # (2*hidden_size) x n_items
        scores = torch.matmul(c_t, B) # batch_size x n_items
        # scores = self.sf(scores)
        return scores

4.4.2. Kiểm tra model NARM

In [ ]:
# Thử nghiệm model bằng cách giả lập 1 input và thực hiện quá trình feed forward
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_size = 3
n_layers = 7
# embedding = nn.Embedding(11000, hidden_size)
input_variable = torch.tensor([[  66,  369,   66, 1272],
                                [ 567,  183,   28,  616],
                                [ 392, 1558, 1143,  175],
                                [ 394,   31,   31, 5558],
                                [   0,    0,    0,    0]]).to(device)

lengths =  torch.tensor([5, 5, 5, 5]).to('cpu')
print('input_seq: \n', input_variable)
print('input_lengths: \n', lengths)
model_test = NARM(hidden_size = hidden_size, n_items  = 100000, embedding_dim = 100, n_layers=1, dropout=0.25).to(device)
print('model phrase: \n', model_test)
scores = model_test.forward(input_seq = input_variable, input_lengths = lengths)
print('probability distribution: ', scores.shape)

input_seq: 
 tensor([[  66,  369,   66, 1272],
        [ 567,  183,   28,  616],
        [ 392, 1558, 1143,  175],
        [ 394,   31,   31, 5558],
        [   0,    0,    0,    0]], device='cuda:0')
input_lengths: 
 tensor([5, 5, 5, 5])
model phrase: 
 NARM(
  (embedding): Embedding(100000, 100, padding_idx=0)
  (gru): GRU(100, 3)
  (emb_dropout): Dropout(p=0.25, inplace=False)
  (a_1): Linear(in_features=3, out_features=3, bias=False)
  (a_2): Linear(in_features=3, out_features=3, bias=False)
  (v_t): Linear(in_features=3, out_features=1, bias=False)
  (ct_dropout): Dropout(p=0.5, inplace=False)
  (b): Linear(in_features=100, out_features=6, bias=False)
  (sf): Softmax(dim=None)
)
probability distribution:  torch.Size([4, 100000])


4.5. Validation
Hàm validation sẽ sử dụng để tính toán recall@k và mrr@k trên tập dữ liệu validation.

Đầu tiên ta sẽ sử dụng model để dự báo xác suất output cho từng batch. valid_loader là một iterator được sử dụng để khởi tạo batch. Sử dụng vòng lặp để để duyệt qua toàn bộ batch của valid_loader và lưu các giá trị recall@k và mrr@k tính được ở mỗi batch vào list. Cuối cùng lấy trung bình của toàn bộ các chỉ số này để thu được recall@k và mrr@k đại diện trên tập validation.

In [ ]:
def validate(valid_loader, model):
    model.eval()
    recalls = []
    mrrs = []
    with torch.no_grad():
        for seq, target, lens in valid_loader:
            seq = seq.to(device)
            target = target.to(device)
            outputs = model(seq, lens)
            logits = F.softmax(outputs, dim = 1)
            recall, mrr = evaluate(logits, target, k = args['topk'])
            recalls.append(recall)
            mrrs.append(mrr)
    
    mean_recall = torch.mean(torch.stack(recalls))
    mean_mrr = torch.mean(torch.stack(mrrs))
    return mean_recall, mean_mrr

4.6. Training model
Hàm main() giúp huấn luyện mô hình trên toàn bộ epochs và trả về kết quả là mô hình sau huấn luyện. Bên cạnh đó, mô hình sau huấn luyện sẽ được lưu vào file latest_checkpoint.pth.tar để có thể tái sử dụng về sau.

Hàm trainForEpoch() thực hiện huấn luyện mô hình trên mỗi một epoch. Dựa vào Data Loader, chúng ta có thể dễ dàng duyệt qua toàn bộ dataset và training trên từng batch.

In [ ]:
import os
import time
import random
import argparse
import pickle
import numpy as np
from tqdm import tqdm
from os.path import join

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn

args = {
    'dataset_path':'../input/yoochoose/yoochoose-clicks.dat',
    'batch_size': 256,
    'hidden_size': 100,
    'embed_dim': 50,
    'epoch': 10,
    'lr':0.001,
    'lr_dc':0.1,
    'lr_dc_step':80,
    'test':None,
    'topk':20,
    'valid_portion':0.1
}

here = os.path.dirname(os.getcwd())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def main():
    print('Loading data...')
    train_data, valid_data, test_data = load_data(train_set=train_index, test_set=test_index)
    train_data = RecSysDataset(train_data)
    valid_data = RecSysDataset(valid_data)
    test_data = RecSysDataset(test_data)
    train_loader = DataLoader(train_data, batch_size = args['batch_size'], shuffle = True, collate_fn = collate_fn)
    valid_loader = DataLoader(valid_data, batch_size = args['batch_size'], shuffle = False, collate_fn = collate_fn)
    test_loader = DataLoader(test_data, batch_size = args['batch_size'], shuffle = False, collate_fn = collate_fn)
    print('Complete load data!')
    n_items = voc.num_items
    model = NARM(hidden_size = args['hidden_size'], n_items = n_items, embedding_dim = args['embed_dim'], n_layers=2, dropout=0.25).to(device)
    print('complete load model!')

    if args['test'] == 'store_true':
        ckpt = torch.load('latest_checkpoint.pth.tar')
        model.load_state_dict(ckpt['state_dict'])
        recall, mrr = validate(test_loader, model)
        print("Test: Recall@{}: {:.4f}, MRR@{}: {:.4f}".format(args['topk'], recall, args['topk'], mrr))
        return model

    optimizer = optim.Adam(model.parameters(), args['lr'])
    criterion = nn.CrossEntropyLoss()
    scheduler = StepLR(optimizer, step_size = args['lr_dc_step'], gamma = args['lr_dc'])

    print('start training!')
    for epoch in tqdm(range(args['epoch'])):
        # train for one epoch
        trainForEpoch(train_loader, model, optimizer, epoch, args['epoch'], criterion, log_aggr = 1000)
        scheduler.step(epoch = epoch)
        recall, mrr = validate(valid_loader, model)
        print('Epoch {} validation: Recall@{}: {:.4f}, MRR@{}: {:.4f} \n'.format(epoch, args['topk'], recall, args['topk'], mrr))

        # store best loss and save a model checkpoint
        ckpt_dict = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        # Save model checkpoint into 'latest_checkpoint2.pth.tar'
        torch.save(ckpt_dict, 'latest_checkpoint.pth.tar')
    return model


def trainForEpoch(train_loader, model, optimizer, epoch, num_epochs, criterion, log_aggr=1000):
    model.train()

    sum_epoch_loss = 0

    start = time.time()
    for i, (seq, target, lens) in enumerate(train_loader):
        seq = seq.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        outputs = model(seq, lens)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step() 

        loss_val = loss.item()
        sum_epoch_loss += loss_val

        iter_num = epoch * len(train_loader) + i + 1

        if i % log_aggr == 0:
            print('[TRAIN] epoch %d/%d  observation %d/%d batch loss: %.4f (avg %.4f) (%.2f im/s)'
                % (epoch + 1, num_epochs, i, len(train_loader), loss_val, sum_epoch_loss / (i + 1),
                  len(seq) / (time.time() - start)))

        start = time.time()

model = main()

Loading data...
--------------------------------------------------
Dataset info:
Number of sessions: 3806780
--------------------------------------------------
--------------------------------------------------
Dataset info:
Number of sessions: 422976
--------------------------------------------------
--------------------------------------------------
Dataset info:
Number of sessions: 416838
--------------------------------------------------
Complete load data!
complete load model!
start training!


  0%|          | 0/10 [00:00<?, ?it/s]

[TRAIN] epoch 1/10  observation 0/14871 batch loss: 10.7536 (avg 10.7536) (19.84 im/s)
[TRAIN] epoch 1/10  observation 1000/14871 batch loss: 8.8231 (avg 9.4189) (1132.23 im/s)
[TRAIN] epoch 1/10  observation 2000/14871 batch loss: 8.4013 (avg 8.9754) (1342.88 im/s)
[TRAIN] epoch 1/10  observation 3000/14871 batch loss: 8.2948 (avg 8.7767) (1189.25 im/s)
[TRAIN] epoch 1/10  observation 4000/14871 batch loss: 8.0520 (avg 8.6459) (1177.64 im/s)
[TRAIN] epoch 1/10  observation 5000/14871 batch loss: 8.0060 (avg 8.5413) (1227.48 im/s)
[TRAIN] epoch 1/10  observation 6000/14871 batch loss: 8.0486 (avg 8.4513) (1301.94 im/s)
[TRAIN] epoch 1/10  observation 7000/14871 batch loss: 7.8767 (avg 8.3647) (1329.70 im/s)
[TRAIN] epoch 1/10  observation 8000/14871 batch loss: 7.4650 (avg 8.2786) (1332.93 im/s)
[TRAIN] epoch 1/10  observation 9000/14871 batch loss: 7.5303 (avg 8.1952) (1099.32 im/s)
[TRAIN] epoch 1/10  observation 10000/14871 batch loss: 7.4389 (avg 8.1143) (1204.57 im/s)
[TRAIN] epoc

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:163: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
 10%|█         | 1/10 [04:44<42:41, 284.61s/it]

Epoch 0 validation: Recall@20: 0.3509, MRR@20: 0.1266 

[TRAIN] epoch 2/10  observation 0/14871 batch loss: 6.6729 (avg 6.6729) (69.26 im/s)
[TRAIN] epoch 2/10  observation 1000/14871 batch loss: 6.7268 (avg 6.7336) (1318.24 im/s)
[TRAIN] epoch 2/10  observation 2000/14871 batch loss: 6.7519 (avg 6.6957) (1221.14 im/s)
[TRAIN] epoch 2/10  observation 3000/14871 batch loss: 6.8528 (avg 6.6656) (1218.58 im/s)
[TRAIN] epoch 2/10  observation 4000/14871 batch loss: 6.7631 (avg 6.6363) (1174.15 im/s)
[TRAIN] epoch 2/10  observation 5000/14871 batch loss: 6.3496 (avg 6.6060) (1210.22 im/s)
[TRAIN] epoch 2/10  observation 6000/14871 batch loss: 6.2374 (avg 6.5812) (1358.86 im/s)
[TRAIN] epoch 2/10  observation 7000/14871 batch loss: 6.4348 (avg 6.5560) (1345.12 im/s)
[TRAIN] epoch 2/10  observation 8000/14871 batch loss: 6.6238 (avg 6.5324) (1198.37 im/s)
[TRAIN] epoch 2/10  observation 9000/14871 batch loss: 6.2324 (avg 6.5095) (1241.48 im/s)
[TRAIN] epoch 2/10  observation 10000/14871 batch

...

Epoch 7 validation: Recall@20: 0.5206, MRR@20: 0.2053 

[TRAIN] epoch 9/10  observation 0/14871 batch loss: 5.5715 (avg 5.5715) (58.63 im/s)
[TRAIN] epoch 9/10  observation 1000/14871 batch loss: 5.4661 (avg 5.5785) (1314.79 im/s)
[TRAIN] epoch 9/10  observation 2000/14871 batch loss: 5.8105 (avg 5.5841) (1247.82 im/s)
[TRAIN] epoch 9/10  observation 3000/14871 batch loss: 5.2520 (avg 5.5844) (1141.99 im/s)
[TRAIN] epoch 9/10  observation 4000/14871 batch loss: 5.7286 (avg 5.5879) (1296.39 im/s)
[TRAIN] epoch 9/10  observation 5000/14871 batch loss: 5.6703 (avg 5.5928) (1164.76 im/s)
[TRAIN] epoch 9/10  observation 6000/14871 batch loss: 5.7791 (avg 5.5956) (1132.05 im/s)
[TRAIN] epoch 9/10  observation 7000/14871 batch loss: 5.8627 (avg 5.5940) (1314.20 im/s)
[TRAIN] epoch 9/10  observation 8000/14871 batch loss: 5.4483 (avg 5.5948) (1332.59 im/s)
[TRAIN] epoch 9/10  observation 9000/14871 batch loss: 5.5932 (avg 5.5958) (1195.41 im/s)
[TRAIN] epoch 9/10  observation 10000/14871 batch

 90%|█████████ | 9/10 [42:37<04:43, 283.08s/it]

Epoch 8 validation: Recall@20: 0.5215, MRR@20: 0.2067 

[TRAIN] epoch 10/10  observation 0/14871 batch loss: 5.2666 (avg 5.2666) (72.99 im/s)
[TRAIN] epoch 10/10  observation 1000/14871 batch loss: 5.7699 (avg 5.5768) (1226.90 im/s)
[TRAIN] epoch 10/10  observation 2000/14871 batch loss: 5.6046 (avg 5.5685) (1215.52 im/s)
[TRAIN] epoch 10/10  observation 3000/14871 batch loss: 5.2650 (avg 5.5685) (1288.95 im/s)
[TRAIN] epoch 10/10  observation 4000/14871 batch loss: 5.7716 (avg 5.5702) (1235.05 im/s)
[TRAIN] epoch 10/10  observation 5000/14871 batch loss: 5.5118 (avg 5.5724) (1345.12 im/s)
[TRAIN] epoch 10/10  observation 6000/14871 batch loss: 5.6683 (avg 5.5735) (1285.08 im/s)
[TRAIN] epoch 10/10  observation 7000/14871 batch loss: 5.5198 (avg 5.5742) (1261.27 im/s)
[TRAIN] epoch 10/10  observation 8000/14871 batch loss: 5.7424 (avg 5.5742) (1127.10 im/s)
[TRAIN] epoch 10/10  observation 9000/14871 batch loss: 5.4600 (avg 5.5752) (1362.70 im/s)
[TRAIN] epoch 10/10  observation 10000/

100%|██████████| 10/10 [47:19<00:00, 283.96s/it]

Epoch 9 validation: Recall@20: 0.5244, MRR@20: 0.2095 



4.7. Dự báo
Trước khi thực hiện dự báo thì ta sẽ cần phải load mô hình từ PATH đã lưu trước đó. Có 2 kiểu save mô hình chính trên pytorch đó là save toàn bộ mô hình và save các checkpoints. Nếu save toàn bộ mô hình sẽ buộc phải xác định các classes và cấu trúc thư mục lưu trữ mô hình nên thường dẫn tới xảy ra lỗi khi thay đổi project. Do đó save theo checkpoint thường được khuyến nghị, đối với bài toán này chúng ta cũng save mô hình theo checkpoint. Để load lại mô hình cũ ta thực hiện như sau:

In [ ]:
import torch
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PATH = 'latest_checkpoint.pth.tar'
model = NARM(hidden_size = args['hidden_size'], n_items = 33751, embedding_dim = args['embed_dim'], n_layers=2, dropout=0.25).to(device)
optimizer = optim.Adam(params = model.parameters(), lr=0.001)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']

model.eval()

NARM(
  (embedding): Embedding(33751, 50, padding_idx=0)
  (gru): GRU(50, 100, num_layers=2)
  (emb_dropout): Dropout(p=0.25, inplace=False)
  (a_1): Linear(in_features=100, out_features=100, bias=False)
  (a_2): Linear(in_features=100, out_features=100, bias=False)
  (v_t): Linear(in_features=100, out_features=1, bias=False)
  (ct_dropout): Dropout(p=0.5, inplace=False)
  (b): Linear(in_features=50, out_features=200, bias=False)
  (sf): Softmax(dim=None)
)

Lưu ý: Sau khi load xong model ta luôn phải chạy hàm model.eval() để kích hoạt các dropout và batchnormalization layer. Nếu không kết quả dự báo có thể dẫn tới sai lệch. Tiếp theo chúng ta sẽ sử dụng mô hình để dự báo cho một trường hợp cụ thể.

In [ ]:
# Lựa chọn ngẫu nhiên một session trên test
import numpy as np
i = np.random.randint(0, len(test_index[0]))
x = [test_index[0][i]]
y = [test_index[1][i]]
print('item indexes sequence input: ', x)
print('item index next output: ', y)

item indexes sequence input:  [[30676, 32394, 29011]]
item index next output:  [27150]


In [ ]:
# Step 1: Khởi tạo test_loader để biến đổi dữ liệu session đưa vào mô hình
test_data = RecSysDataset([x, y])
test_loader = DataLoader(test_data, batch_size = args['batch_size'], shuffle = False, collate_fn = collate_fn)

# Step 2: Dự báo các indice tiếp theo mà khách hàng có khả năng click
def _preddict(loader, model):
    model.eval()
    recalls = []
    mrrs = []
    j = 1
    with torch.no_grad():
      for seq, target, lens in loader:
        seq = seq.to(device)
        target = target.to(device)
        outputs = model(seq, lens)
        logits = F.softmax(outputs, dim = 1)
        _, indices = torch.topk(logits, 20, -1)
        print('Is next clicked item in top 20 suggestions: ', (target in indices))
        print('Top 20 next item indices suggested: ')
    return indices

_preddict(test_loader, model)

--------------------------------------------------
Dataset info:
Number of sessions: 1
--------------------------------------------------
Is next clicked item in top 20 suggestions:  False
Top 20 next item indices suggested: 


tensor([[30675, 32346, 30690, 29010, 29008, 29056, 29024, 29020, 29043, 28984,
         28562, 27093, 27098, 29027, 29011, 30693, 32401, 32358, 28985, 29048]],
       device='cuda:0')